In [12]:
# TODO from insights by prof
# dont save in json but locally in a var
# only match by labels (no other matching needed, no properties, superclasses, etc.)
# just save labels in list with key is class uri and value is label: if memory is not a problem save in to structures: key value of label and value is class uri
# 5 - 10 min for human, mouse ontologies is reasonable

# Output: in rdf format (see reference_anatomy) just put final matches in there above the threshold (defined by the user). As relation use owl:equivalentClass

In [13]:
# Required libraries
!pip install sentence_transformers
!pip install pandas
!pip install rdflib
!pip install python-Levenshtein
!pip install scikit-learn


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


# Project Description
The goal of the project is to develop a simple yet effective ontology alignment framework in Python that focuses on lexical similarity matching. The framework will utilize both string matching techniques and the semantic capabilities of large language models to identify potential alignments between entities (such as classes) in two different ontologies.

### Objectives
1. **Develop an ontology alignment framework** that can process and compare ontologies based on textual content.
2. **Implement lexical similarity matching** using both basic string matching techniques and advanced semantic analysis with embeddings from LLMs.
3. **Output alignments with confidence scores**, enabling users to understand and evaluate the quality and reliability of the suggested alignments.

### Steps to Perform

#### Step 1: Ontology Parsing
- **Goal**: Load and parse the ontologies to be aligned.
- **Tasks**:
  - Utilize libraries like `rdflib` or `owlready2` to read ontology files.
  - Extract relevant textual information (e.g., class names, labels, descriptions).

#### Step 2: Lexical Similarity Matching
This step is divided into two sub-steps: string matching and embeddings matching.

##### a. String Matching
- **Goal**: Implement direct and fuzzy string comparison techniques to find matches based on textual similarity.
- **Tasks**:
  - Perform normalization (e.g., lowercasing, removing special characters).
  - Use string comparison methods (exact match, substring search, edit distance).

##### b. Embeddings Matching Using LLMs
- **Goal**: Use the semantic context provided by LLMs to match terms based on their meanings.
- **Tasks**:
  - Generate embeddings for the textual content of each ontology using models from the Hugging Face Transformers library.
  - Calculate similarity scores between embeddings (e.g., using cosine similarity).

#### Step 3: Combining and Filtering
- **Goal**: Aggregate results from both matching techniques and refine the output.
- **Tasks**:
  - Combine scores from string and embeddings matching.
  - Apply thresholds to filter out matches with low confidence.
  - Optionally, use simple structural checks to add confidence to matches (e.g., matched entities have similar parent classes).

#### Step 4: Output and Evaluation
- **Goal**: Output the alignment results and provide means for evaluation.
- **Tasks**:
  - Format the output in a structured way (e.g., JSON, CSV) that lists entity pairs and their matching scores.
  - If possible, evaluate the effectiveness using known benchmarks or test cases to calculate precision, recall, and F1-score.

### Summary
The project is centered on creating a practical tool for ontology matching, focusing on textual content using both conventional and advanced NLP techniques. By combining string-based and semantic-based approaches, the framework aims to provide robust alignments that are supported by both literal and contextual text similarities. This dual approach enhances the capability of the alignment process, making it more flexible and potentially more accurate than using only one method.

In [14]:
# imports
import json
import rdflib
import pandas as pd
from collections import OrderedDict, defaultdict

**rdflib vs owlready2:**

Interchangeability: Given that OWL is an application of RDF, tools that can parse RDF/XML can generally handle .owl files, and vice versa, provided that the ontology-specific constructs are understood by the tool. This is why libraries like rdflib, which are capable of parsing RDF, are suitable for handling OWL files serialized in RDF/XML format.

Flexibility: Choosing to work with rdflib for general RDF handling and owlready2 for specific ontology manipulations where needed is a flexible approach. It allows you to leverage the strengths of both libraries—rdflib for its robust RDF manipulation and SPARQL querying capabilities, and owlready2 for its ontology-specific features like reasoning and direct manipulation of classes and properties.

# Load/ Parse ontologies

In [15]:
# input paths
onto1_path_in = "test_ontologies/mouse.owl"
onto2_path_in = "test_ontologies/human.owl"


In [16]:
def load_ontology(file_path):
    """
    Loads an ontology from a given file path, which can be in RDF (.rdf) or OWL (.owl) format.

    Args:
    file_path (str): The file path to the ontology file.

    Returns:
    rdflib.Graph: A graph containing the ontology data.
    """
    # Create a new RDF graph
    graph = rdflib.Graph()

    # Bind some common namespaces to the graph
    namespaces = {
        "rdf": rdflib.namespace.RDF,
        "rdfs": rdflib.namespace.RDFS,
        "owl": rdflib.namespace.OWL,
        "xsd": rdflib.namespace.XSD
    }
    for prefix, namespace in namespaces.items():
        graph.namespace_manager.bind(prefix, namespace)

    # Attempt to parse the file
    try:
        graph.parse(file_path, format=rdflib.util.guess_format(file_path))
        print(f"Successfully loaded ontology from {file_path}")
    except Exception as e:
        print(f"Failed to load ontology from {file_path}: {e}")
        return None

    return graph

In [17]:
# load ontologies
onto1_graph = load_ontology(onto1_path_in)
onto2_graph = load_ontology(onto2_path_in)
print(onto1_graph, onto2_graph)

Successfully loaded ontology from test_ontologies/mouse.owl
Successfully loaded ontology from test_ontologies/human.owl
[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory']]. [a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory']].


In [18]:
def preprocess_label(label):
    return str(label).replace("_", " ").strip(" ,.").lower()

### New approach without json and instead dicts

In [19]:
def extract_ontology_details_to_dict(graph):
    # Query for classes
    class_query = """
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?class ?label ?label_dt ?label_lang
    WHERE {
        ?class rdf:type owl:Class.
        OPTIONAL { ?class rdfs:label ?label. BIND(datatype(?label) AS ?label_dt) BIND(lang(?label) AS ?label_lang) }
    }
    """
    classes = graph.query(class_query)
    ontology_labels_dict = OrderedDict()
    labels_list = []

    # Process class results
    for row in classes:
        class_uri, label, label_dt, label_lang = row
        class_key = str(class_uri)
        label_str = preprocess_label(label)

        if label_str is None or label_str == "none":
            continue

        if label_str not in ontology_labels_dict:
            ontology_labels_dict[label_str] = class_key
            labels_list.append(label_str)

    return ontology_labels_dict, labels_list

In [20]:
onto1_dict, onto1_list = extract_ontology_details_to_dict(onto1_graph)
onto2_dict, onto2_list = extract_ontology_details_to_dict(onto2_graph)

In [21]:
onto1_dict

OrderedDict([('mouse anatomy', 'http://mouse.owl#MA_0000001'),
             ('spinal cord grey matter', 'http://mouse.owl#MA_0000002'),
             ('organ system', 'http://mouse.owl#MA_0000003'),
             ('trunk', 'http://mouse.owl#MA_0000004'),
             ('body cavity/lining', 'http://mouse.owl#MA_0000005'),
             ('head/neck', 'http://mouse.owl#MA_0000006'),
             ('limb', 'http://mouse.owl#MA_0000007'),
             ('tail', 'http://mouse.owl#MA_0000008'),
             ('adipose tissue', 'http://mouse.owl#MA_0000009'),
             ('cardiovascular system', 'http://mouse.owl#MA_0000010'),
             ('connective tissue', 'http://mouse.owl#MA_0000011'),
             ('endocrine system', 'http://mouse.owl#MA_0000012'),
             ('hemolymphoid system', 'http://mouse.owl#MA_0000013'),
             ('integumental system', 'http://mouse.owl#MA_0000014'),
             ('muscle', 'http://mouse.owl#MA_0000015'),
             ('nervous system', 'http://mouse.owl#

In [22]:
onto1_list

['mouse anatomy',
 'spinal cord grey matter',
 'organ system',
 'trunk',
 'body cavity/lining',
 'head/neck',
 'limb',
 'tail',
 'adipose tissue',
 'cardiovascular system',
 'connective tissue',
 'endocrine system',
 'hemolymphoid system',
 'integumental system',
 'muscle',
 'nervous system',
 'sensory organ',
 'skeletal system',
 'visceral organ system',
 'back',
 'abdomen/pelvis/perineum',
 'thorax',
 'head',
 'neck',
 'forelimb',
 'hindlimb',
 'lower back',
 'upper back',
 'abdomen',
 'pelvis',
 'chest',
 'thoracic cavity',
 'arm',
 'lower arm',
 'upper arm',
 'elbow',
 'hand',
 'shoulder',
 'wrist',
 'carpus',
 'hand digit',
 'metacarpus',
 'ankle',
 'foot',
 'hip',
 'knee',
 'leg',
 'foot digit',
 'metatarsus',
 'tarsus',
 'lower leg',
 'upper leg',
 'pericardial cavity',
 'peritoneal cavity',
 'pleural cavity',
 'fat',
 'brown fat',
 'white fat',
 'blood',
 'blood vessel',
 'arterial blood vessel',
 'aorta',
 'arteriole',
 'artery',
 'capillary',
 'venous blood vessel',
 'vein',


In [23]:
# def find_duplicates(ordered_dict): # => filtering for multiple labels works => see outdated notebook

## 3. Matching

### 3.1. String Matching

For String Matching we will implement 4 different methods that the user then can chose via a parameter when calling the method.

The metrics we will use are:
- Levenshtein distance
- Jaccard Similarity
- Cosine Similarity
- TF-IDF

In [24]:
import Levenshtein
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score

def levenshtein_distance(str1, str2):
    return Levenshtein.distance(str1, str2)

def calc_cosine_similarity(str1, str2):
    vectorizer = CountVectorizer()
    count_matrix = vectorizer.fit_transform([str1, str2])
    return cosine_similarity(count_matrix)[0][1]

def jaccard_similarity(str1, str2):
    # Tokenize the strings into sets of words
    set1 = set(str1.split())
    set2 = set(str2.split())

    # Find the intersection and union of the two sets
    intersection = set1.intersection(set2)
    union = set1.union(set2)

    # Calculate the Jaccard score
    if not union:  # Handle the edge case where both strings might be empty
        return 0.0
    return len(intersection) / len(union)

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def cosine_vectorize_labels(labels):
    """
    Converts a list of labels into TF-IDF vectors using TfidfVectorizer.

    Args:
    labels (list): List of all labels from both ontologies.

    Returns:
    TfidfVectorizer, scipy.sparse.csr.csr_matrix: The vectorizer and the TF-IDF matrix.
    """
    vectorizer = CountVectorizer()
    count_matrix = vectorizer.fit_transform(labels)
    return vectorizer, count_matrix

def cosine_compare_labels(count_matrix, index1, index2):
    """
    Computes the cosine similarity between two labels based on their count vector indices.

    Args:
    count_matrix (scipy.sparse.csr.csr_matrix): The matrix containing the count vectors.
    index1, index2 (int): Indices of the labels to compare.

    Returns:
    float: Cosine similarity score.
    """
    return cosine_similarity(count_matrix[index1:index1+1], count_matrix[index2:index2+1])[0][0]


def execute_cosine_string_matching(label_list1, label_list2):
    # Combine labels and vectorize them
    all_labels = label_list1 + label_list2
    vectorizer, count_matrix = cosine_vectorize_labels(all_labels)

    # Example comparison between the first label of ontology 1 and the first label of ontology 2
    similarity_score = cosine_compare_labels(count_matrix, 0, len(label_list1))
    print(f"Similarity score between '{label_list1[0]}' and '{label_list2[0]}': {similarity_score}")

In [26]:
execute_cosine_string_matching(onto1_list, onto2_list)

Similarity score between 'mouse anatomy' and 'anatomic structure system or substance': 0.0


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def tfidf_vectorize_labels(labels):
    """
    Converts a list of labels into TF-IDF vectors using TfidfVectorizer.

    Args:
    labels (list): List of all labels from both ontologies.

    Returns:
    TfidfVectorizer, scipy.sparse.csr.csr_matrix: The vectorizer and the TF-IDF matrix.
    """
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(labels)
    return vectorizer, tfidf_matrix

def tfidf_compare_labels(tfidf_matrix, index1, index2):
    """
    Computes the cosine similarity between two labels based on their TF-IDF vector indices.

    Args:
    tfidf_matrix (scipy.sparse.csr.csr_matrix): The matrix containing the TF-IDF vectors.
    index1, index2 (int): Indices of the labels to compare.

    Returns:
    float: Cosine similarity score.
    """
    return cosine_similarity(tfidf_matrix[index1:index1+1], tfidf_matrix[index2:index2+1])[0][0]


def execute_tfidf_string_matching(label_list1, label_list2):
    # Combine labels and vectorize them
    all_labels = label_list1 + label_list2
    vectorizer, tfidf_matrix = tfidf_vectorize_labels(all_labels)

    # Example comparison between the first label of ontology 1 and the first label of ontology 2
    similarity_score = tfidf_compare_labels(tfidf_matrix, 0, len(label_list1))
    print(f"Similarity score between '{label_list1[0]}' and '{label_list2[0]}': {similarity_score}")

In [28]:
execute_tfidf_string_matching(onto1_list, onto2_list)

Similarity score between 'mouse anatomy' and 'anatomic structure system or substance': 0.0


In [29]:
def execute_string_matching(metric, data1, data2):
    """
    Executes the selected matching metric on the provided data.

    Args:
    metric (str): A single letter representing the metric to use.
                  'Levenshtein' for Levenshtein Distance,
                  'Jaccard' for Jaccard Similarity,
                  'LinkTransformer' for Link Transformer.
    data1, data2 (str): The data strings to compare.

    Returns:
    result: The result of the chosen metric computation.
    """
    if metric == 'Levenshtein':
        return levenshtein_distance(data1, data2)
    elif metric == 'Jaccard':
        return jaccard_similarity(data1, data2)
    elif metric == 'LinkTransformer':
        pass # TODO implement or remove
    else:
        raise ValueError("Invalid metric selection")

In [30]:
len(onto1_dict)

2738

In [31]:
# load ontologies
onto1_graph = load_ontology(onto1_path_in)
onto2_graph = load_ontology(onto2_path_in)

onto1_dict, onto1_list = extract_ontology_details_to_dict(onto1_graph)
onto2_dict, onto2_list = extract_ontology_details_to_dict(onto2_graph)

print(len(onto1_list))
print(len(onto2_list))

Successfully loaded ontology from test_ontologies/mouse.owl
Successfully loaded ontology from test_ontologies/human.owl
2738
3298


In [32]:
def match_ontologies(onto1_path_in, onto2_path_in, metric, bidirectional=False):
    # load ontologies
    onto1_graph = load_ontology(onto1_path_in)
    onto2_graph = load_ontology(onto2_path_in)

    onto1_dict, onto1_list = extract_ontology_details_to_dict(onto1_graph)
    onto2_dict, onto2_list = extract_ontology_details_to_dict(onto2_graph)

    labels_already_tested_labels = {} # dict to store when labels (of ontology 2) were already tested for label (of ontology 1) => necessary to avoid infite loop

    for label in onto1_list:
        labels_already_tested_labels[label] = []

    onto2_used_classes = {}

    if metric == "Cosine":
        execute_cosine_string_matching(onto1_list, onto2_list)
        # TODO extend so it return scores etc. and also can be mapped to nodes for next step
    elif metric == "TF-IDF":
        execute_tfidf_string_matching(onto1_list, onto2_list)
        # TODO extend so it return scores etc. and also can be mapped to nodes for next step
    else:
        class_results = {}

        while onto1_list: # loop over labels of ontology 1 until empty
            #print(len(onto1_list))
            label1 = onto1_list.pop() # remove the last element in the list => removing the last (instead of first) makes things easier and less error prone
            # labels that got added again cause a better match was found (see later step) will be appended to the end and therefore handled immediately

            # Match from Ontology 1 to Ontology 2
            label_result = [label1, "", 0]
            best_score = 0
            already_tested_labels = labels_already_tested_labels[label1]
            for label2 in onto2_list:
                if label2 not in already_tested_labels: # check that label wasn't already checked in previous run
                    matching_score = execute_string_matching(metric, label1, label2) # calculate string matching score
                    # If a perfect match is found, stop iterating over labels for this entry
                    if (metric == 'Jaccard' and matching_score == 1) or (metric == 'Levenshtein' and matching_score == 0): # handle perfect match
                        best_score = matching_score
                        label_result = [label1, label2, best_score]
                        break # stop searching for matches cause perfect match found
                    # Check if a match for this label has been found before
                    if (metric == 'Jaccard' and matching_score > best_score) or (metric == 'Levenshtein' and matching_score < best_score): # handle higher score then before
                        label_result[1] = label2
                        best_score = matching_score

            label_result[2] = best_score # save best score in label_result
            label_with_best_score = label_result[1] # get label that achieved the best score

            class_uri = onto1_dict[label1] # get the class_uri of the currently checked label in ontology 1
            if label_result[2] == 0 and label_with_best_score == '': # handle if no match was found
                class_results[class_uri] = label_result
            else:
                class2_uri = onto2_dict[label_with_best_score] # get class_uri of the label with best match
                label_result[1] = class2_uri # save class_uri instead of label => TODO maybe change to not manipulate label_result as it is confusing for later steps

                if class2_uri not in onto2_used_classes: # check if class found of ontology 2 is NOT already used by other class in ontology 1
                    if class_uri not in class_results: # handle no entry exists yet for that class
                        class_results[class_uri] = label_result
                        onto2_used_classes[class2_uri] = class_uri
                        labels_already_tested_labels[label1].append(label_with_best_score)
                    elif label_result[2] > class_results[class_uri][2]: # handle entry exist but now higher score was found with another label of the class (handles multiple labels)
                            class_results[class_uri] = label_result
                            onto2_used_classes[class2_uri] = class_uri
                            labels_already_tested_labels[label1].append(label_with_best_score)
                else: # class of ontology 2 already in use
                    result_current_class_in_use = onto2_used_classes[class2_uri] # get class uri of class that uses that class of ontology 2
                    if label_result[2] > class_results[result_current_class_in_use][2]: # if score of the new found match is higher than the current assigned one
                        class_results[class_uri] = label_result # set the class of ontology 2 to that current class
                        onto2_used_classes[class2_uri] = class_uri # overwrite the use of that class to new class of ontology 1
                        old_used_label = class_results[result_current_class_in_use][0]
                        labels_already_tested_labels[old_used_label].append(label_with_best_score)
                        onto1_list.append(old_used_label) # add the old used label again to list again that gets iterated as it now doesnt have a match anymore
                        class_results[result_current_class_in_use] = ["", "", 0] # set result of earlier class to None (could also be remmoved but that way later we can handle if no match found)
                    else: # handle not a higher score
                        labels_already_tested_labels[label1].append(label_with_best_score) # add the label to the already_tested_labels
                        onto1_list.append(label1) # append currently check label again as it needs to handled again with the new information of already_tested_labels

        # TODO implement bidirectional matching

        # OLD_TO-DO important: currently it takes the best match found for the current class of the ontology 1.
        # But it doesnt take into account if a later class has a higher score with that class and therefore would be better suited
        # Solution: make dict and always take one element that gets removed. If later another element matches with the class matched with
        # the previous element the earlier removed element get added again and the value of that elements gets assigned to the higher value element

        return class_results


In [33]:
# input paths
onto1_path_in = "test_ontologies/mouse.owl"
onto2_path_in = "test_ontologies/human.owl"

matching_results = match_ontologies(onto1_path_in, onto2_path_in, 'Jaccard')

Successfully loaded ontology from test_ontologies/mouse.owl
Successfully loaded ontology from test_ontologies/human.owl


In [34]:
len(matching_results)
# TODO one element is always missing: length of classes in onto_1 is 2739 but resulting matches is 2738

2737

In [35]:
# Comparing with reference_anatomy where matches all have measure of 1
print(matching_results["http://mouse.owl#MA_0000110"])
print(matching_results["http://mouse.owl#MA_0001468"])
print(matching_results["http://mouse.owl#MA_0001886"])
print(matching_results["http://mouse.owl#MA_0000702"]) # has 0.75
print(matching_results["http://mouse.owl#MA_0000751"]) # has 0.8
print(matching_results["http://mouse.owl#MA_0000064"])
print(matching_results["http://mouse.owl#MA_0000062"])
print(matching_results["http://mouse.owl#MA_0002497"]) # has 0.66
print(matching_results["http://mouse.owl#MA_0001221"])
print(matching_results["http://mouse.owl#MA_0001424"]) # has 0.25 => check why so low and for which class then the correct label was used: http://human.owl#NCI_C32242

['intervertebral disc', 'http://human.owl#NCI_C49571', 1.0]
['occipital bone', 'http://human.owl#NCI_C12757', 1.0]
['external ear cartilage', 'http://human.owl#NCI_C49225', 1.0]
['aorta smooth muscle', 'http://human.owl#NCI_C49191', 0.75]
['lymphatic vessel smooth muscle', 'http://human.owl#NCI_C49260', 0.8]
['artery', 'http://human.owl#NCI_C12372', 1.0]
['aorta', 'http://human.owl#NCI_C12669', 1.0]
['liver perisinusoidal space', 'http://human.owl#NCI_C33309', 0.6666666666666666]
['tensor tympani', 'http://human.owl#NCI_C33748', 1.0]
['cervical vertebra 4', 'http://human.owl#NCI_C33501', 0.25]


In [37]:
import itertools
import time
from sentence_transformers import SentenceTransformer, util

def calculate_label_similarity_llm(model_name, onto1_dict, onto2_dict):
  """
  Calculate cosine similarity between pairs of labels from two sets and return the results in a dictionary.
  Each key in the dictionary is the class URI from ontology 1, and each value is a list of tuples,
  each containing the label from ontology 2, the class URI from ontology 2, and the similarity score.

  Parameters:
  model_name (str): Name of the Sentence Transformer model to be used.
  onto1_dict (OrderedDict): Dictionary where keys are labels and values are class URIs for the first ontology.
  onto2_dict (OrderedDict): Dictionary where keys are labels and values are class URIs for the second ontology.

  Returns:
  dict: A dictionary with class URIs from the first ontology as keys and lists of tuples (label, class URI, score) from the second ontology as values.
  """
  model = SentenceTransformer(model_name, device='cpu')
  onto1_labels, onto1_classes = zip(*onto1_dict.items())
  onto2_labels, onto2_classes = zip(*onto2_dict.items())

  onto1_label_embeddings = model.encode(list(onto1_labels), convert_to_tensor=True)
  onto2_label_embeddings = model.encode(list(onto2_labels), convert_to_tensor=True)

  similarity_scores = util.pytorch_cos_sim(onto1_label_embeddings, onto2_label_embeddings)

  # Initialize the dictionary to hold results
  results_dict = {}

  # Fill the dictionary with similarity scores
  for i, onto1_class in enumerate(onto1_classes):
    results_dict[onto1_class] = {}
    for j, onto2_class in enumerate(onto2_classes):
      results_dict[onto1_class][onto2_class] = similarity_scores[i][j].item()

  # Sort the dictionary entries by similarity score within each onto1_class
  sorted_results_dict = {}
  for onto1_class in results_dict:
    sorted_onto2_classes = sorted(results_dict[onto1_class].items(), key=lambda x: x[1], reverse=True)
    sorted_results_dict[onto1_class] = dict(sorted_onto2_classes)

  return sorted_results_dict

# Record start time
start_time = time.time()

dict_similarity_scores_llm = calculate_label_similarity_llm("sentence-transformers/all-MiniLM-L12-v2", onto1_dict, onto2_dict)

# Record end time
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time

print(f"Time taken: {elapsed_time} seconds")

/Users/tildietrich/.pyenv/versions/3.10.12/envs/kg_ontology_matching_3.10.12/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Time taken: 40.48931813240051 seconds


In [38]:
# Print the first entries from the dictionary
for onto1_class in list(dict_similarity_scores_llm.keys())[:20]:
    print(f"{onto1_class}: {dict_similarity_scores_llm[onto1_class]}")

http://mouse.owl#MA_0000001: {'http://human.owl#NCI_C28460': 0.5620852708816528, 'http://human.owl#NCI_C12934': 0.5187564492225647, 'http://human.owl#NCI_C12958': 0.5052818059921265, 'http://human.owl#NCI_C33218': 0.47970739006996155, 'http://human.owl#NCI_C12646': 0.473985493183136, 'http://human.owl#NCI_C13052': 0.4669102728366852, 'http://human.owl#NCI_C52701': 0.4637256860733032, 'http://human.owl#NCI_C32977': 0.45982518792152405, 'http://human.owl#NCI_C52703': 0.4525844156742096, 'http://human.owl#NCI_C33774': 0.4472803473472595, 'http://human.owl#NCI_C12433': 0.4464266002178192, 'http://human.owl#NCI_C41409': 0.4459545314311981, 'http://human.owl#NCI_C12644': 0.4436717629432678, 'http://human.owl#NCI_C52714': 0.44338420033454895, 'http://human.owl#NCI_C13111': 0.4416525065898895, 'http://human.owl#NCI_C32472': 0.44156646728515625, 'http://human.owl#NCI_C12465': 0.4390401542186737, 'http://human.owl#NCI_C12435': 0.43798378109931946, 'http://human.owl#NCI_C52717': 0.435526460409164

In [39]:
def set_new_match(class1_uri, class2_uri, score, onto2_used_classes, class_results):
    """Set a new match for class1_uri and class2_uri."""
    onto2_used_classes[class2_uri] = class1_uri
    class_results[class1_uri] = ["", class2_uri, "", score]  # label is empty for now

def update_matching(new_class1_uri, class2_uri, new_score, old_class1_uri, onto2_used_classes, class_results, onto1_class_list):
    """Update matches when a better score is found."""
    onto2_used_classes[class2_uri] = new_class1_uri
    class_results[old_class1_uri] = ["", "", "", 0]  # Clear old match
    class_results[new_class1_uri] = ["", class2_uri, "", new_score]
    onto1_class_list.append(old_class1_uri)

def reevaluate(class1_uri, class2_uri, already_tested_classes, onto1_class_list):
    """Re-add class1_uri for re-evaluation."""
    onto1_class_list.append(class1_uri)
    already_tested_classes[class1_uri].add(class2_uri)

def perform_matching_llm(dict_similarity_scores_llm):
    # Initialize dictionaries and lists
    already_tested_classes = {}
    class_results = {}
    onto2_used_classes = {}
    onto1_class_list = list(dict_similarity_scores_llm.keys())

    while onto1_class_list:
        class1_uri = onto1_class_list.pop()
        already_tested_classes[class1_uri] = already_tested_classes.get(class1_uri, set())

        # Iterate over each class2_uri and score from the pre-sorted dictionary
        for class2_uri, score in dict_similarity_scores_llm[class1_uri].items():
            if class2_uri not in already_tested_classes[class1_uri]:
                already_tested_classes[class1_uri].add(class2_uri)  # Mark this class2_uri as tested

                if score >= 0.99:  # Check for a perfect match
                    set_new_match(class1_uri, class2_uri, score, onto2_used_classes, class_results)
                    break  # Found a perfect match, skip further checks for this class1_uri

                # If no perfect match, check if it's not already linked
                if class2_uri not in onto2_used_classes:
                    set_new_match(class1_uri, class2_uri, score, onto2_used_classes, class_results)
                    break  # Successfully linked, no need to continue

                # If already linked, check if the new score is better
                elif score > class_results[onto2_used_classes[class2_uri]][3]:
                    old_class1_uri = onto2_used_classes[class2_uri]
                    update_matching(class1_uri, class2_uri, score, old_class1_uri, onto2_used_classes, class_results, onto1_class_list)
                    break  # Updated the link, no need to continue
            else:
                # This class2_uri was already checked, continue to the next
                continue

    return class_results

# Call function
matched_results_llm = perform_matching_llm(dict_similarity_scores_llm)

In [40]:
# Print the first entries from the dictionary
for onto1_class in list(matched_results_llm.keys())[:20]:
    print(f"{onto1_class}: {matched_results_llm[onto1_class]}")

http://mouse.owl#MA_0002758: ['', 'http://human.owl#NCI_C53167', '', 0.6024026274681091]
http://mouse.owl#MA_0002757: ['', 'http://human.owl#NCI_C41416', '', 0.6656939387321472]
http://mouse.owl#MA_0002756: ['', 'http://human.owl#NCI_C12969', '', 0.7062833905220032]
http://mouse.owl#MA_0002755: ['', 'http://human.owl#NCI_C32951', '', 0.765652060508728]
http://mouse.owl#MA_0002754: ['', 'http://human.owl#NCI_C33714', '', 0.3939228355884552]
http://mouse.owl#MA_0002753: ['', 'http://human.owl#NCI_C12967', '', 0.6740885972976685]
http://mouse.owl#MA_0002752: ['', 'http://human.owl#NCI_C12451', '', 0.7293311953544617]
http://mouse.owl#MA_0002751: ['', 'http://human.owl#NCI_C12895', '', 0.7586371302604675]
http://mouse.owl#MA_0002750: ['', 'http://human.owl#NCI_C33588', '', 0.9309330582618713]
http://mouse.owl#MA_0002749: ['', 'http://human.owl#NCI_C12464', '', 0.7721498012542725]
http://mouse.owl#MA_0002748: ['', 'http://human.owl#NCI_C13082', '', 1.0000004768371582]
http://mouse.owl#MA_00

In [41]:
def transform_dict(original_dict):
    """
    Transforms a dictionary where URLs are the values into a dictionary where
    URLs are the keys and the values are concatenated labels associated with each URL.

    Args:
    original_dict (OrderedDict): The original dictionary with labels as keys and URLs as values.

    Returns:
    dict: A dictionary with URLs as keys and concatenated labels as values.
    """
    new_dict = {}
    for label, url in original_dict.items():
        if url in new_dict:
            new_dict[url] += " " + label  # Concatenating labels with space; change as needed
        else:
            new_dict[url] = label
    return new_dict

onto1_transformed_dict = transform_dict(onto1_dict)
onto2_transformed_dict = transform_dict(onto2_dict)

In [42]:
# Add labels
def add_labels(data, onto1_label, onto2_label):
    # Iterate through each key and update the list with labels
    for key, values in data.items():
        # First empty string in the list gets replaced by the label from onto1_label if available
        label1 = onto1_label.get(key)
        if label1 is not None:
            values[0] = label1

        # Second empty string in the list gets replaced by the label from onto2_label using the second entry's key if available
        label2_key = values[1]  # The second entry in the list is assumed to be a key for the onto2_label
        label2 = onto2_label.get(label2_key)
        if label2 is not None:
            values[2] = label2

    return data

matched_results_llm_with_labels = add_labels(matched_results_llm, onto1_transformed_dict, onto2_transformed_dict)

In [43]:
# Print the first entries from the dictionary
for onto1_class in list(matched_results_llm_with_labels.keys())[:20]:
    print(f"{onto1_class}: {matched_results_llm_with_labels[onto1_class]}")

http://mouse.owl#MA_0002758: ['levator auris longus', 'http://human.owl#NCI_C53167', 'longissimus cervicis', 0.6024026274681091]
http://mouse.owl#MA_0002757: ['periolivary nucleus', 'http://human.owl#NCI_C41416', 'perineural cell', 0.6656939387321472]
http://mouse.owl#MA_0002756: ['lateral spinal nucleus', 'http://human.owl#NCI_C12969', 'descending spinal cord tract', 0.7062833905220032]
http://mouse.owl#MA_0002755: ['lateral cervical nucleus', 'http://human.owl#NCI_C32951', 'lateral vestibular nucleus', 0.765652060508728]
http://mouse.owl#MA_0002754: ['neocortex', 'http://human.owl#NCI_C33714', 'sylvian cistern', 0.3939228355884552]
http://mouse.owl#MA_0002753: ['spinal cord intermediolateral nucleus', 'http://human.owl#NCI_C12967', 'ascending spinal cord tract', 0.6740885972976685]
http://mouse.owl#MA_0002752: ['nucleus approprius', 'http://human.owl#NCI_C12451', 'caudate nucleus', 0.7293311953544617]
http://mouse.owl#MA_0002751: ['spinal cord ventral column', 'http://human.owl#NCI_C

In [ ]:
# check for http://mouse.owl#MA_0001424
# http://mouse.owl#MA_0001424 has label cervical vertebra 4
# currently matched with http://human.owl#NCI_C32245 => has label C7_Vertebra
# but in the reference.rdf matched with http://human.owl#NCI_C32242 => has label C4_Vertebra with score 1
# REPORT when running the matching again the http://mouse.owl#MA_0001424 gets matched to http://human.owl#NCI_C32431
# => which has a score of 0.16 => reason: because now the elements where iterated differently the before matched label got now matched with
# another class but with same score 0.25 => http://mouse.owl#MA_0001436: ['sacral vertebra 3', 'http://human.owl#NCI_C32245', 0.25]
print(jaccard_similarity("cervical vertebra 4", "C4_Vertebra"))
print(jaccard_similarity("thoracic vertebra 10", "C4_Vertebra"))

0.0
0.0


In [ ]:
counter = 0
unique_values = set()
for key, element in matching_results.items():
    if element[1] == "http://human.owl#NCI_C32245":
        print(key)
        print(element)
    if not element[2] == 0 and not element[2] == 1:
        counter += 1
        unique_values.add(element[2])
# unique_values

http://mouse.owl#MA_0001443
['thoracic vertebra 6', 'http://human.owl#NCI_C32245', 0.25]


In [ ]:
# TODO implement combining and filtering
# TODO implement user inputs (also weighted average with user defined formular would be nice)

Output format example:
```
<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF xmlns="http://knowledgeweb.semanticweb.org/heterogeneity/alignment"
	 xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
	 xmlns:xsd="http://www.w3.org/2001/XMLSchema#">

<Alignment>
<xml>yes</xml>
<level>0</level>
<type>??</type>

<map>
	<Cell>
		<entity1 rdf:resource="http://mouse.owl#MA_0002401"/>
		<entity2 rdf:resource="http://human.owl#NCI_C52561"/>
		<measure rdf:datatype="xsd:float">1.0</measure>
		<relation>=</relation>
	</Cell>
</map>
```